## ЗАДАНИЕ ДЛЯ ПРОГРАММИРУЮЩИХ

**Ссылка**, на источник текста

In [1]:
DATA_URL = "http://az.lib.ru/t/tolstoj_a_k/text_0180.shtml"

Устанавливаем библиотеки

In [2]:
import warnings
warnings.filterwarnings('ignore')
!pip install -q git+https://github.com/dvolchek/rnnmorph.git

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 990.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 46.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


Создаём объект морфологического анализатора `RNNMorph`

In [3]:
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")

Скачиваем текст, по которому будет дано задание, с помощью `urllib`

In [4]:
import urllib.request

opener = urllib.request.URLopener({})
resource = opener.open(DATA_URL)
raw_text = resource.read().decode(resource.headers.get_content_charset()) #Текс с html тегами

In [5]:
raw_text[:200]

'<html>\r\n<head>\r\n<title>Lib.ru/Классика: Толстой Алексей Константинович. Семья вурдалака</title>\r\n</head>\r\n\r\n<body>\r\n\r\n\r\n<center>\r\n\r\n<h2><a href=/t/tolstoj_a_k/>Толстой Алексей Константинович</a><br>\r\n'

Как видно, текст содержит html теги, от которых нужно избавиться. Выбрасываем из текста HTML-теги с помощью библиотеки Beatiful soap

In [6]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(raw_text, features="html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
cleaned_text = soup.get_text()

In [7]:
cleaned_text[:200]

'\n\nLib.ru/Классика: Толстой Алексей Константинович. Семья вурдалака\n\n\n\nТолстой Алексей Константинович\r\nСемья вурдалака\n\n\nLib.ru/Классика:\n\r\n\n\n[Регистрация]\n \n\r\n\r\n\r\n[Найти] \r\n[Рейтинги]\r\n[Обсуждения]\r\n['

С помощью библиотеки [NLTK](https://nltk.org/) разбиваем текст на предложения и токены.

In [8]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

tokenized_sentences = [word_tokenize(sentence) for sentence in sent_tokenize(cleaned_text)]
"A total of %d 'sentences'" % len(tokenized_sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


"A total of 576 'sentences'"

## Задание 1
С помощью метода `str.isalpha` из стандартной библиотеки Python модифицируйте нижеследующий код так, чтобы в predictions остались только буквенные токены.

In [11]:
from tqdm import tqdm
predictions = [[pred.normal_form for pred in sent if pred.normal_form.isalpha()]
               for sent in tqdm(predictor.predict_sentences(sentences=tokenized_sentences), "sentences") ]
predictions[-11:-10] #Сейчас видно, что токены типа "точка", "запятая" и тд пока присутствуют в предложениях. От них нужно избавиться



9/9 [==============================] - 6s 685ms/step



sentences: 100%|██████████| 576/576 [00:00<00:00, 48757.20it/s]


[['другой',
  'ребёнок',
  'я',
  'такой',
  'же',
  'образ',
  'кинуть',
  'вслед',
  'но',
  'он',
  'упасть',
  'прямо',
  'под',
  'копыто',
  'лошадь',
  'и',
  'быть',
  'раздавленный']]

Проверьте себя. Должны получиться следующие значения:

*   Предложений: 577 (возможны расхождения в несколько предложений)
*   Токенов: примерно 8621 (возможны расхождения в некоторое количество токенов)

In [12]:
len(predictions)

576

In [13]:
non_uniq_tokens = [word for sentence in predictions for word in sentence]
len(non_uniq_tokens)


8619

Для продолжения работы над заданием числа должны быть близки к указанным

## Задание 2

Используя `non_uniq_tokens`, стоп-слова для русского языка из библиотеки nltk (`nltk.corpus.stopwords`) и `nltk.FreqDist`, вычислите, **какую долю среди 100 самых частотных** токенов в произведении занимают токены, **не относящиеся** к стоп словам.

**Например**, если среди 100 самых частотных слов встречается 25 слов, входящих в стоп лист, значит не входят в стоп лист 75 слов, и их доля составит 0.75.

**Не бойтесь использовать документацию NLTK и тьюториалы.**

In [14]:
import nltk
from nltk import FreqDist
from nltk.corpus import stopwords
nltk.download("stopwords")
STOPWORDS = set(stopwords.words("russian"))
stopwords.words("russian")[:5] #Пример стоп слов

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['и', 'в', 'во', 'не', 'что']

In [15]:
freq = FreqDist(non_uniq_tokens)

In [20]:
freq100 = freq.most_common(100)
len(freq100)

100

In [23]:
ans = sum(1 for w in freq100 if w[0] not in STOPWORDS)/100
ans

0.49

Проверьте себя: должно получиться 0.49 (допустимо небольшое расхождение)

## Задание 3
Вычислите, сколько токенов встречается в тексте **строго больше** 50 раз.

In [24]:
res = sum(1 for val in freq.values() if val > 50)
res

22

Проверьте себя: должно получиться значение 22 (возможно небольшое расхождение)


# Задание из Блока 1

http://az.lib.ru/n/nekrasow_n_a/text_1840_pevitza.shtml

In [25]:
DATA_URL = "http://az.lib.ru/n/nekrasow_n_a/text_1840_pevitza.shtml"

In [26]:
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")

sentences:   0%|          | 0/576 [26:06<?, ?it/s]


Получаем текст без лишнего

In [27]:
import urllib.request
from bs4 import BeautifulSoup

opener = urllib.request.URLopener({})
resource = opener.open(DATA_URL)
raw_text = resource.read().decode(resource.headers.get_content_charset()) #Текс с html тегами

soup = BeautifulSoup(raw_text, features="html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
cleaned_text = soup.get_text()
cleaned_text[:200]

'\n\nLib.ru/Классика: Некрасов Николай Алексеевич. Певица\n\n\n\nНекрасов Николай Алексеевич\r\nПевица\n\n\nLib.ru/Классика:\n\r\n\n\n[Регистрация]\n \n\r\n\r\n\r\n[Найти] \r\n[Рейтинги]\r\n[Обсуждения]\r\n[Новинки]\r\n[Обзоры]\r\n[Пом'

Разбиваем на предложения и токены

In [28]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

tokenized_sentences = [word_tokenize(sentence) for sentence in sent_tokenize(cleaned_text)]
"A total of %d 'sentences'" % len(tokenized_sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


"A total of 911 'sentences'"

Количество токенов

In [29]:
from tqdm import tqdm
predictions = [[pred.normal_form for pred in sent if pred.normal_form.isalpha()]
               for sent in tqdm(predictor.predict_sentences(sentences=tokenized_sentences), "sentences") ]
predictions[-11:-10]

15/15 [==============================] - 8s 430ms/step


sentences: 100%|██████████| 911/911 [00:00<00:00, 131995.68it/s]


[['с']]

In [30]:
print(f"Кол-во предложений: {len(predictions)}")
non_uniq_tokens = [word for sentence in predictions for word in sentence]
print(f"Кол-во токенов: {len(non_uniq_tokens)}")


Кол-во предложений: 911
Кол-во токенов: 11695


Какую долю среди 40 самых частотных токенов в произведении занимают слова, не входящие в стоп-лист?

In [31]:
import nltk
from nltk import FreqDist
from nltk.corpus import stopwords
nltk.download("stopwords")
STOPWORDS = set(stopwords.words("russian"))

freq = FreqDist(non_uniq_tokens)
freq40 = freq.most_common(40)

ans = sum(1 for w in freq40 if w[0] not in STOPWORDS)/40
ans

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0.325

Сколько токенов встречается в тексте строго больше 20 раз?

In [32]:
res = sum(1 for val in freq.values() if val > 20)
res

76